In [1]:
from rnndissect.utils.model_utils import predict_sentiment
import rnndissect.utils.nlp_utils as nlpu
import rnndissect.activations.extractor as extr
import sys
import torch
import pickle

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

sys.path.append("../model")
from lstm import LSTM
from configs import *

with open("/home/goncalo/Documents/rnndissect/assets/imdb_vocab.pkl", "rb") as vf:
    vocab = pickle.load(vf)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
sentence = "The movie ends an era of the franchise by taking a second stab at adapting a classic comics arc, with deeply disappointing results."

### Create and import the model weights

In [3]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = False
DROPOUT = 0
PAD_IDX = vocab.stoi['pad']

config = LSTM_CONFIG1
config.output_dim = 2
model = LSTM(25002, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)
model.load_state_dict(torch.load("../model/lstmo.pt"))
model.to(DEVICE)

LSTM(
  (embedding): Embedding(25002, 100, padding_idx=7635)
  (rnn): LSTM(100, 256, num_layers=2)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0, inplace=False)
)

In [4]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    logit = model(tensor, length_tensor)
    prediction = torch.sigmoid(logit)
    print(logit)
    return prediction.item()

## Generate a random initial input

In [5]:
# shape should be n_words x embedding_dim
# how many n_words to optimize the output? try different lengths

min_emb, max_emb = model.embedding.weight.min().item(), model.embedding.weight.max().item()

def generate_random_input(shape, min_lim, max_lim):
    """shape is a pair tuple
       uniform distribution is used"""
    return torch.FloatTensor(shape[0], shape[1]).uniform_(min_lim, max_lim)

### Activation maximization begins

In [6]:
def am_loop(lr, x, model, thresh):
    diff = float("inf")
    iters = 0
    model.train()
    
    lstm_out, _ = model.rnn(x.view(1, 1, -1))
    prev_logits = model.fc(lstm_out)
    prev_logits.backward()
    
    while diff > thresh:
        iters += 1
        x.requires_grad = False
        x = x + lr * x.grad
        x.requires_grad = True
        
        lstm_out, _ = model.rnn(x.view(1, 1, -1))
        logits = model.fc(lstm_out)
        logits.backward()
        diff = (logits - prev_logits).sum().item()
        prev_logits = logits
    print(iters)
    print(logits)
    return x

In [7]:
x = generate_random_input((1, 100), min_emb, max_emb).to(DEVICE)
x.requires_grad = True

In [8]:
res_list = []

for _ in range(5):
    x = generate_random_input((1, 100), min_emb, max_emb).to(DEVICE)
    x.requires_grad = True
    res_list.append(am_loop(0.01, x, model, 1e-6)[0].cpu().detach().numpy())

137824
tensor([[[4.0134]]], device='cuda:0', grad_fn=<AddBackward0>)
174798
tensor([[[4.1065]]], device='cuda:0', grad_fn=<AddBackward0>)
124806
tensor([[[4.1561]]], device='cuda:0', grad_fn=<AddBackward0>)
179007
tensor([[[4.0760]]], device='cuda:0', grad_fn=<AddBackward0>)
141605
tensor([[[4.2206]]], device='cuda:0', grad_fn=<AddBackward0>)


In [12]:
predict_sentiment(model, "Wyatt")

tensor([[-0.4575]], device='cuda:0', grad_fn=<AddmmBackward>)


0.38757142424583435

In [66]:
def index_of_max(lst):
    return lst.index(min(lst))

def get_n_most_similar(vec, vecs, vocab, n=10):
    words = []
    sims = []
    best_idx = 0
    best_sim = 0
    worst_sim_idx = 0
    for i in range(len(vecs)):
        curr_sim = cosine_similarity([vec], [vecs[i].numpy()])
        if len(words) == 10:
            if curr_sim > sims[worst_sim_idx]:
                words[worst_sim_idx] = i
                sims[worst_sim_idx] = curr_sim
                worst_sim_idx = index_of_max(sims)
        else:
            words.append(i)
            sims.append(curr_sim)
    return list(vocab.itos[i] for i in words)

In [68]:
get_n_most_similar(vocab.vectors[vocab.stoi["bullet"]].numpy(), vocab.vectors, vocab)

['riddled',
 'grenade',
 'wound',
 'bullets',
 'chest',
 'stab',
 'shotgun',
 'bullet',
 'gunshot',
 'wounds']